In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display
import pickle
import torch

In [2]:
import sys
import os

# Add the 'project' directory to the path
sys.path.append(os.path.abspath('..'))

from project_code.data.load_data import load_col_types, load_data
from project_code.data.prepare_data_sklearn import get_features_targets
from project_code.utils.results import get_best_model_file
from project_code.inference.parameters import get_core_parameter_predictions, convert_output_to_parameter_predictions, PARAMETER_COLS

# Loading data and models

In [3]:
datasets_folder = f'../data/processed/'

dataset_of_model = {
    'SRTaxo1NN': 'biologist_no_pub_age',  
    'Taxo1NN': 'biologist_no_pub_age',  
    'MLP': 'final_taxonomy_ecocodes',
    'DEBNetHC': 'final_taxonomy_ecocodes',
    'DEBNetSC': 'final_taxonomy_ecocodes',
}


## Loading best models

In [4]:
def load_model(model_file, results_folder):
    if model_file[-4:] == '.pkl':
        with open(f"{results_folder}/models/{model_file}", 'rb') as f:
            model = pickle.load(f)
    elif model_file[-4:] == '.pth':
        model = torch.load(f"{results_folder}/models/{model_file}")
        model.eval()
    return model

In [5]:
best_models = {}
for mt in dataset_of_model.keys():  
    results_folder = f'../results/{dataset_of_model[mt]}'
    metric = 'GEF'
    model_file = get_best_model_file(results_folder=results_folder, model_type=mt, metric=metric)
    if model_file is not None:
        best_models[mt] = load_model(model_file, results_folder)
print(best_models.keys())

dict_keys(['SRTaxo1NN', 'MLP'])


C:\Users\diogo\AppData\Local\Temp\ipykernel_10288\4173759641.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(f"{results_folder}/models/{model_file}")


## Loading data

In [6]:
all_dfs = {}
all_col_types = {}
all_data = {}
for dataset_name in list(set(dataset_of_model.values())):
    results_folder = f'../results/{dataset_name}'
    all_dfs[dataset_name] = load_data(dataset_name=dataset_name, data_split='train_test', datasets_folder=datasets_folder)
    all_col_types[dataset_name] = load_col_types(dataset_name=dataset_name, datasets_folder=datasets_folder)
    if 'biologist' not in dataset_name:
        all_data[dataset_name] = get_features_targets(all_dfs[dataset_name], all_col_types[dataset_name])
    else:
        model = best_models['SRTaxo1NN']
        encoded_dfs = {}
        # Encoded data with trained model encoders
        for split in ('train', 'test'):
            encoded_dfs[split] = model.regressor.encode_data(all_dfs[dataset_name][split])
        all_data[dataset_name] = get_features_targets(data=encoded_dfs, col_types=all_col_types[dataset_name])

# Visualize predictions

In [7]:
taxonomy_cols = [col for col in all_col_types['final_taxonomy_ecocodes']['input']['all'] if 'class' in col]

hue_series = {
    'metamorphosis': pd.concat([all_dfs['biologist_no_pub_age'][data_split]['metamorphosis'] for data_split in ['train', 'test']]),
    'class': pd.concat([pd.from_dummies(all_dfs['final_taxonomy_ecocodes'][data_split][taxonomy_cols], sep='_') for data_split in ['train', 'test']])['class'],
    'climate': None,
    'habitat': None,
    'migrate': None,
    'food': None
}

hue_orders = {
    'metamorphosis': [False, True],
    'class': None,
    'climate': None,
    'habitat': None,
    'migrate': None,
    'food': None
}


In [8]:
def plot_residuals_df(model_type, plot_kind, data_split, groupby, scale):
    # Get data for the model
    dataset_name = dataset_of_model[model_type]
    data = all_data[dataset_name]
    col_types = all_col_types[dataset_name]
    model = best_models[model_type]

    # Get predictions:
    y_true_ps = data[data_split]['output']
    if 'DEBNet' in model_type:
        y_pred_ps = model.predict(torch.tensor(data[data_split]['input'], dtype=torch.float32))
    elif model_type == 'TaxonomicKNNRegressor' and data_split == 'train':
        train_distance_matrix = model.regressor_._compute_distance_matrix(model.regressor_.train_data, model.regressor_.train_data, data_split='train')
        y_hat, indices = model.regressor_.get_predictions_from_distance_matrix(train_distance_matrix)
        y_pred_ps = model.regressor_apply_scaling_relationships(model.regressor_.train_data, y_hat, indices)

    else:
        y_pred_ps = model.predict(data[data_split]['input'])

    if scale == 'model':
        cols_to_plot = col_types['output']['all']
        target_df = pd.DataFrame(y_true_ps, columns=col_types['output']['all'])
        pred_df = pd.DataFrame(y_pred_ps, columns=col_types['output']['all'])
        
    elif scale == 'parameter':
        cols_to_plot = PARAMETER_COLS            
        target_df = convert_output_to_parameter_predictions(y_true_ps, data[data_split]['input'], col_types)
        pred_df = convert_output_to_parameter_predictions(y_pred_ps, data[data_split]['input'], col_types)

    target_df.set_index(all_dfs[dataset_name][data_split].index, inplace=True)
    pred_df.set_index(all_dfs[dataset_name][data_split].index, inplace=True)
    if plot_kind == 'residual_vs_predicted':
        if scale == 'model':
            residuals_df = target_df - pred_df
        elif scale == 'parameter':
            residuals_df = (target_df - pred_df) / target_df  

    # Plot predictions vs targets  
    n_cols = 3
    n_rows = np.ceil(len(cols_to_plot) / n_cols).astype(int)
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(16, 5*n_rows), tight_layout=True)
    fig.suptitle(model_type, fontsize=16)
    margin_factor = 0.05
    for i, col in enumerate(cols_to_plot):
        ax = axes[i // n_cols, i % n_cols]

        if scale == 'parameter' and col != 'kap':
            plot_log_scale_x = True
            if plot_kind == 'residual_vs_predicted':
                plot_log_scale_y = False
            else:
                plot_log_scale_y = True
        elif scale == 'model' and col in col_types['output']['log']:
            plot_log_scale_x = True
            plot_log_scale_y = True
        else:
            plot_log_scale_x = False
            plot_log_scale_y = False
        
        if plot_kind == 'residual_vs_predicted':
            sns.scatterplot(x=pred_df[col], y=residuals_df[col], ax=ax, hue=hue_series[groupby], hue_order=hue_orders[groupby]) # Fix
            min_v = (1-margin_factor)*min(target_df[col].min(), pred_df[col].min())
            max_v = (1+margin_factor)*max(target_df[col].max(), pred_df[col].max())
            ax.set_xlim([min_v, max_v])

            ax.plot([min_v, max_v], [0, 0], 'k--')
            ax.set_ylabel('Residuals (actual - predicted)')

        elif plot_kind == 'actual_vs_predicted':
            sns.scatterplot(x=pred_df[col], y=target_df[col], ax=ax, hue=hue_series[groupby], hue_order=hue_orders[groupby])
            min_v = (1-margin_factor)*min(target_df[col].min(), pred_df[col].min())
            max_v = (1+margin_factor)*max(target_df[col].max(), pred_df[col].max())
            ax.set_xlim([min_v, max_v])
            ax.set_ylim([min_v, max_v])
            ax.plot([min_v, max_v], [min_v, max_v], 'k--')
            ax.set_ylabel('Actual values')

        if plot_log_scale_x:
            ax.set_xscale('log')
        else:
            ax.set_xscale('linear')
        
        if plot_log_scale_y:
            ax.set_yscale('log')
        else:
            ax.set_yscale('linear')

        ax.set_xlabel('Predicted values') 
        #r2 = metrics.r2_score(target_df, pred_df)
        ax.set_title(f"{col}")

model_selector = widgets.Dropdown(options=list(best_models.keys()), value='MLP', description='Model:')
plot_selector = widgets.Dropdown(options=['actual_vs_predicted', 'residual_vs_predicted'], value='actual_vs_predicted', description='Plot Type:')
data_split_selector = widgets.Dropdown(options=['train', 'test'], value='test', description='Data Split: ')
groupby_selector = widgets.Dropdown(options=['metamorphosis', 'class', 'climate', 'habitat', 'migrate', 'food'])
scale_selector = widgets.Dropdown(options=['model', 'parameter',], value='parameter')
widgets.interactive(plot_residuals_df, model_type=model_selector, plot_kind=plot_selector, data_split=data_split_selector, groupby=groupby_selector, scale=scale_selector)

interactive(children=(Dropdown(description='Model:', index=1, options=('SRTaxo1NN', 'MLP'), value='MLP'), Drop…

# Save parameter predictions

### AmP values

In [9]:
dataset_name = 'biologist_no_pub_age'
dfs = all_dfs[dataset_name]
col_types = all_col_types[dataset_name]
gt_df = pd.concat({ds: dfs[ds][col_types['output']['all']] for ds in ('train', 'test')}).reset_index(level=0, names='data_split')
gt_pars_df = get_core_parameter_predictions(dfs, pred_df=gt_df, col_types=col_types)
gt_pars_df.to_csv(f'../results/parameter_predictions/AmP_predictions.csv', float_format='%.6e')
gt_pars_df

,data_split,z,p_M,kap,v,E_G,E_Hb,E_Hx,E_Hj,E_Hp,k_J
species,,,,,,,,,,,
Sepiella_inermis,train,0.444090,882.4408,0.73830,0.025902,5483.824,56.740000,56.740000,14830.00000,16330.000,0.002000
Peucaea_botterii,train,1.701899,584.5174,0.97544,0.048308,7320.566,177.200000,742.900000,177.20000,1937.000,0.023876
Molothrus_bonariensis,train,2.013300,678.5390,0.82945,0.055504,7307.254,2226.000000,13000.000000,2226.00000,31500.000,0.028023
Xiphophorus_maculatus,train,0.964880,287.8024,0.57892,0.022653,5223.355,12.040000,12.040000,12.04000,1646.000,0.002000
Cnidoglanis_macrocephalus,train,9.231700,30.9836,0.53392,0.025475,5211.104,148.300000,148.300000,171.20000,579500.000,0.002000
...,...,...,...,...,...,...,...,...,...,...,...
Eremophilus_mutisii,test,3.225900,73.4373,0.98486,0.046142,5222.791,0.019530,0.019530,0.04938,534.200,0.002000
Atropus_atropos,test,1.268300,16.5963,0.91326,0.019594,5232.015,0.018900,0.018900,3.13600,35150.000,0.002000
Heteronotia_binoei,test,1.260900,32.9960,0.89733,0.007429,7913.671,101.100000,101.100000,101.10000,1126.000,0.002000


### ML models

In [12]:
def save_parameter_predictions(model_type):
    model = best_models[model_type]
    dataset_name = dataset_of_model[model_type]
    dfs = all_dfs[dataset_name]
    data = all_data[dataset_name]
    col_types = all_col_types[dataset_name]
    pred_df = pd.DataFrame()
    for split in ('train', 'test'):
        if model_type == 'TaxonomicKNNRegressor' and split == 'train':
            train_distance_matrix = model.regressor_._compute_distance_matrix(model.regressor_.train_data, model.regressor_.train_data, data_split='train')
            y_hat, indices = model.regressor_.get_predictions_from_distance_matrix(train_distance_matrix)
            y_pred = model.regressor_apply_scaling_relationships(model.regressor_.train_data, y_hat, indices)
        else:
            y_pred = model.predict(data[split]['input'])
        split_pred_df = pd.DataFrame(data=y_pred, index=dfs[split].index, columns=col_types['output']['all'])
        split_pred_df['data_split'] = split
        pred_df = pd.concat([pred_df, split_pred_df])
    pars_df = get_core_parameter_predictions(dfs, pred_df=pred_df, col_types=col_types)
    predictions_file_name = f'../results/parameter_predictions/{model_type}_predictions.csv'
    pars_df.to_csv(predictions_file_name, float_format='%.10e')
    print(f'Saved predictions for model {model_type} in {predictions_file_name}')

    return pars_df

In [13]:
for model_type in best_models:
    save_parameter_predictions(model_type)

Saved predictions for model SRTaxo1NN in ../results/parameter_predictions/SRTaxo1NN_predictions.csv
Saved predictions for model MLP in ../results/parameter_predictions/MLP_predictions.csv


## Finding the bug in feasibility for TaxonomicKNNRegressor

# Compare parameter predictions

Table with the MAPE for each parameter in each line

In [ ]:
def get_mape(model_type, data_split):
    model = best_models[model_type]
    dataset_name = dataset_of_model[model_type]
    dfs = all_dfs[dataset_name]
    data = all_data[dataset_name]
    col_types = all_col_types[dataset_name]
    pred_df = pd.DataFrame()
    
    y_true_ps = data[data_split]['output']
    if 'DEBNet' in model_type:
        y_pred_ps = model.predict(torch.tensor(data[data_split]['input'], dtype=torch.float32))
    elif model_type == 'TaxonomicKNNRegressor' and data_split == 'train':
        train_distance_matrix = model.regressor_._compute_distance_matrix(model.regressor_.train_data, model.regressor_.train_data, model.regressor_.train_weights,
                                                          model.regressor_.train_weights, data_split='train')
        y_pred_ps = model.regressor_.get_predictions_from_distance_matrix(train_distance_matrix)
    else:
        y_pred_ps = model.predict(data[data_split]['input'])
            
    return compute_parameter_mape(y_true=y_true_ps, y_pred=y_pred_ps, output_col_names=col_types['output']['all'])  


In [ ]:
model_list = ['DEBNetHCSoftplus', 'DEBNet', 'DEBNetSC', 'TaxonomicKNNRegressor']
mape_df = pd.DataFrame(index=model_list, columns=PARAMETER_COLS)
for model_type in model_list:
    model_mape = get_mape(model_type=model_type, data_split='test')
    mape_df.loc[model_type] = model_mape['mean_absolute_percentage_error']
mape_df

In [ ]:
model_labels = {
    'DEBNet': 'DEBNet',
    'DEBNetHCSoftplus': 'DEBNet + HC',
    'DEBNetSC': 'DEBNet + SC',
    'TaxonomicKNNRegressor': 'Taxonomic 1-NN',
    
}

In [ ]:
#for mt, row in mape_df.iterrows():
for mt, label in model_labels.items():
    row = mape_df.loc[mt]
    line = f"{label} "
    for par in PARAMETER_COLS:
        mape = row[par]
        if mape == min(mape_df[par]):
            line += f' & \\textbf{{ {mape:.4f} }}'
        else:
            line += f' & {mape:.4f}'
    line += ' \\\\'
    print(line)